In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/yyk22004396/universal-innovation-training-set/kaggle
/kaggle/input/datasets/yyk22004396/universal-innovation-training-set/json example.txt
/kaggle/input/datasets/yyk22004396/universal-innovation-training-set/kaggle_upload/patent_expert_final.jsonl
/kaggle/input/datasets/yyk22004396/universal-innovation-training-set/kaggle_upload/patent_innovation_metadata.jsonl


In [2]:
# # ============================================================
# # Purpose: Fine-tune T5 with EXPLICITLY INTERPRETABLE LATENT SPACE
# # Stability: Anti-deadlock version for 640k+ dataset
# # Hardware: Optimized for NVIDIA P100 (16GB)
# # ============================================================

# !pip install -q transformers datasets peft accelerate torch

# import os
# import json
# import torch
# from datasets import Dataset
# from transformers import (
#     T5Tokenizer,
#     T5ForConditionalGeneration,
#     Seq2SeqTrainer,
#     Seq2SeqTrainingArguments,
#     DataCollatorForSeq2Seq,
# )
# from peft import LoraConfig, get_peft_model, TaskType

# # ================= 1. 配置 =================
# class Config:
#     MODEL_NAME = "Langboat/mengzi-t5-base"
#     TRAIN_FILE = "/kaggle/input/datasets/yyk22004396/universal-innovation-training-set/kaggle_upload/patent_expert_final.jsonl"
#     OUTPUT_DIR = "/kaggle/working/model_output"

#     MAX_SOURCE_LEN = 384
#     MAX_TARGET_LEN = 512

#     # P100 优化：12 * 5 = 60 等效 Batch Size
#     BATCH_SIZE = 12
#     GRAD_ACC = 5
#     EPOCHS = 3          # 大数据量下 3 轮即可收敛并形成流形
#     LR = 2e-4           # 略微调低学习率，增加训练稳定性

#     USE_LORA = True

# config = Config()

# # ================= 2. 数据加载 (流式读取防止 RAM 崩溃) =================
# def load_data(fp):
#     data = []
#     print(f"📖 Loading data from {fp}...")
#     with open(fp, "r", encoding="utf-8") as f:
#         for line in f:
#             try:
#                 data.append(json.loads(line))
#             except:
#                 continue
#     return Dataset.from_list(data)

# # ================= 3. 数据预处理 =================
# def preprocess_fn(examples, tokenizer):
#     model_inputs = tokenizer(
#         examples["input_text"],
#         max_length=config.MAX_SOURCE_LEN,
#         truncation=True,
#         padding=False,
#     )

#     labels = tokenizer(
#         examples["target_text"],
#         max_length=config.MAX_TARGET_LEN,
#         truncation=True,
#         padding=False,
#     )["input_ids"]

#     model_inputs["labels"] = [
#         [(l if l != tokenizer.pad_token_id else -100) for l in lab]
#         for lab in labels
#     ]
#     return model_inputs

# # ================= 4. 训练核心 =================
# def train():
#     tokenizer = T5Tokenizer.from_pretrained(config.MODEL_NAME)
#     model = T5ForConditionalGeneration.from_pretrained(config.MODEL_NAME)

#     # LoRA 结构保护
#     if config.USE_LORA:
#         lora_cfg = LoraConfig(
#             task_type=TaskType.SEQ_2_SEQ_LM,
#             r=16,
#             lora_alpha=32,
#             lora_dropout=0.05,
#             target_modules=["q", "v"]
#         )
#         model = get_peft_model(model, lora_cfg)
#         model.print_trainable_parameters()

#     # 数据集准备
#     raw_dataset = load_data(config.TRAIN_FILE)
    
#     # 💥 防死锁关键：将验证集限制在 1000 条，不再按百分比
#     dataset = raw_dataset.train_test_split(test_size=1000)

#     print("🛠️ Preprocessing dataset...")
#     tokenized = dataset.map(
#         lambda x: preprocess_fn(x, tokenizer),
#         batched=True,
#         batch_size=1000,
#         remove_columns=dataset["train"].column_names,
#         desc="Running tokenizer"
#     )

#     # 训练参数
#     args = Seq2SeqTrainingArguments(
#         output_dir=config.OUTPUT_DIR,
#         # 💥 稳定性优化
#         evaluation_strategy="steps", # 改为按步数评估
#         eval_steps=1000,             # 每 1000 步看一眼验证集，避免 Epoch 结尾大规模计算死锁
#         logging_steps=10,            # 极高频率打印日志，防止界面判定死锁
        
#         learning_rate=config.LR,
#         per_device_train_batch_size=config.BATCH_SIZE,
#         per_device_eval_batch_size=config.BATCH_SIZE,
#         gradient_accumulation_steps=config.GRAD_ACC,
#         num_train_epochs=config.EPOCHS,
#         fp16=True,                   # 必开，加速并省显存
#         save_strategy="steps",
#         save_steps=2000,
#         save_total_limit=1,
#         load_best_model_at_end=True,
#         metric_for_best_model="eval_loss",
#         greater_is_better=False,
#         report_to="none"             # 禁用外部报告防止网络 IO 导致阻塞
#     )

#     trainer = Seq2SeqTrainer(
#         model=model,
#         args=args,
#         train_dataset=tokenized["train"],
#         eval_dataset=tokenized["test"],
#         tokenizer=tokenizer,
#         data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
#     )

#     print("🚀 Training started. Monitor the log below...")
#     trainer.train()

#     # 最终保存
#     final_dir = "/kaggle/working/final_model"
#     trainer.save_model(final_dir)
#     tokenizer.save_pretrained(final_dir)
#     print(f"✅ Success! Model saved to {final_dir}")

# if __name__ == "__main__":
#     train()

In [3]:
# ====== 关键修复：先解决依赖冲突（必须按顺序安装） ======
!apt-get update -y
!apt-get install -y build-essential

# 1. 升级 huggingface_hub 到兼容版本 (>=0.21.0)
!pip install -q --upgrade "huggingface_hub==0.21.0"

# 2. 安装兼容的 tokenizers (0.22.0) - 与 transformers==4.15.0 兼容
!pip install -q --only-binary=:all: "tokenizers==0.22.0"

# 3. 安装 transformers (4.15.0) - 与 tokenizers 0.22.0 兼容
!pip install -q "transformers==4.15.0"

# 4. 安装其他依赖
!pip install -q datasets peft accelerate torch

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,361 kB]
Get:12 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:13 https://r2u.

In [4]:
# ==============================
# 快速检查核心库版本
# ==============================
core_libs = [
    "torch", "torchvision", "torchaudio",
    "transformers", "tokenizers", "huggingface_hub",
    "peft", "accelerate", "datasets",
    "numpy", "pandas", "scikit-learn"
]

print("🔍 核心库版本检查:")
for lib in core_libs:
    try:
        version = pkg_resources.get_distribution(lib).version
        print(f"✅ {lib}: {version}")
    except Exception as e:
        print(f"❌ {lib}: NOT INSTALLED")

🔍 核心库版本检查:
❌ torch: NOT INSTALLED
❌ torchvision: NOT INSTALLED
❌ torchaudio: NOT INSTALLED
❌ transformers: NOT INSTALLED
❌ tokenizers: NOT INSTALLED
❌ huggingface_hub: NOT INSTALLED
❌ peft: NOT INSTALLED
❌ accelerate: NOT INSTALLED
❌ datasets: NOT INSTALLED
❌ numpy: NOT INSTALLED
❌ pandas: NOT INSTALLED
❌ scikit-learn: NOT INSTALLED


In [5]:
# ============================================================
# T5多任务训练 - 创新方法服务模型
# 环境: Kaggle P100 (16GB VRAM, 13GB RAM, 12h限制)
#
# 修复清单:
#   [1] 自动安装缺失依赖 (peft, accelerate)
#   [2] 辅助任务采样控制 防止训练超时
#   [3] Dataset不含metadata字段 节省~2GB内存
#   [4] 合并LoRA后保存完整T5模型 确保推理服务可加载
#   [5] 训练后提取语义空间 与metadata文件索引对齐
# ============================================================

import subprocess
import sys
import os

# ====== 安装Kaggle环境缺失的依赖 ======
for _pkg in ["peft", "accelerate"]:
    try:
        __import__(_pkg)
    except ImportError:
        print(f"Installing {_pkg}...")
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-q", _pkg]
        )

import json
import random
import torch
import numpy as np
import pickle
from datasets import Dataset
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.decomposition import PCA


# ================= 1. 配置 =================
class Config:
    MODEL_NAME = "Langboat/mengzi-t5-base"

    # 数据路径 (Kaggle dataset) — 根据实际dataset名称调整
    TRAIN_FILE = "/kaggle/input/datasets/yyk22004396/universal-innovation-training-set/kaggle_upload/patent_expert_final.jsonl"
        
    
    METADATA_FILE = "/kaggle/input/datasets/yyk22004396/universal-innovation-training-set/kaggle_upload/patent_innovation_metadata.jsonl"
        

    # 输出路径
    OUTPUT_DIR = "/kaggle/working/model_output"
    FINAL_MODEL_DIR = "/kaggle/working/final_model"
    SEMANTIC_DIR = "/kaggle/working/semantic_analysis"

    # 序列长度
    MAX_SOURCE_LEN = 128
    MAX_TARGET_LEN = 256

    # 训练超参 (P100优化)
    BATCH_SIZE = 16       # P100 16GB稳定运行
    GRAD_ACC = 4          # 有效batch = 64
    EPOCHS = 1
    LR = 2e-4
    WARMUP_STEPS = 500

    # LoRA
    USE_LORA = True
    LORA_R = 16
    LORA_ALPHA = 32

    # ------ 多任务控制 ------
    # 辅助任务采样比例 (0.0 ~ 1.0)
    #   0.1  = 每种辅助任务取10%  (~增加30%训练时间)
    #   0.0  = 仅主任务           (与原代码时间相同)
    # 如果Kaggle超时，优先降低此值
    AUX_SAMPLE_RATIO = 0.15

    # 语义空间提取样本数 (训练后提取)
    SEMANTIC_MAX_SAMPLES = 5000


config = Config()


# ================= 2. 数据加载 =================
def load_training_data():
    """
    加载训练数据 + 元数据, 生成多任务训练样本.
    关键: Dataset中不存储metadata字典, 只保留训练所需的3个字段.
    """
    # --- 检查文件 ---
    if not os.path.exists(config.TRAIN_FILE):
        raise FileNotFoundError(
            f"Training file not found: {config.TRAIN_FILE}\n"
            "Please check your Kaggle dataset path."
        )

    # --- 读取训练数据 (保留None占位以维持行号对齐) ---
    print(f"Loading: {config.TRAIN_FILE}")
    train_items = []
    with open(config.TRAIN_FILE, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                train_items.append(None)
                continue
            try:
                train_items.append(json.loads(line))
            except json.JSONDecodeError:
                train_items.append(None)

    n_valid = sum(1 for x in train_items if x is not None)
    print(f"  Training samples: {n_valid:,}")

    # --- 读取元数据 (可选, 用于辅助任务) ---
    meta_items = []
    has_meta = os.path.exists(config.METADATA_FILE)

    if has_meta:
        print(f"Loading: {config.METADATA_FILE}")
        with open(config.METADATA_FILE, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    meta_items.append(None)
                    continue
                try:
                    meta_items.append(json.loads(line))
                except json.JSONDecodeError:
                    meta_items.append(None)
        n_meta = sum(1 for x in meta_items if x is not None)
        print(f"  Metadata samples: {n_meta:,}")
    else:
        print(f"  Metadata not found, training main task only")

    # --- 构建多任务样本 ---
    main_samples = []
    aux_fop = []
    aux_contradiction = []
    aux_onto = []

    for idx in range(len(train_items)):
        item = train_items[idx]
        if item is None:
            continue

        # 主任务: Input -> Solution (全部保留)
        main_samples.append({
            "input_text": item["input_text"],
            "target_text": item["target_text"],
            "label": item.get("label", ""),
        })

        # 辅助任务需要元数据
        meta = meta_items[idx] if idx < len(meta_items) else None
        if meta is None:
            continue

        # 辅助1: FOP三要素
        try:
            fop = meta["innovation_method"]["fop"]
            parts = []
            funcs = fop.get("function", [])
            objs = fop.get("object", [])
            procs = fop.get("process", [])
            if funcs:
                parts.append(f"Function: {', '.join(funcs[:3])}")
            if objs:
                parts.append(f"Object: {', '.join(objs[:5])}")
            if procs:
                parts.append(f"Process: {', '.join(procs[:3])}")
            if parts:
                aux_fop.append({
                    "input_text": item["input_text"],
                    "target_text": " | ".join(parts),
                    "label": "FOP",
                })
        except (KeyError, TypeError):
            pass

        # 辅助2: 矛盾参数
        try:
            c = meta["contradiction"]
            improve = c.get("improve", "")
            worsen = c.get("worsen", "")
            if improve and worsen:
                aux_contradiction.append({
                    "input_text": item["input_text"],
                    "target_text": f"Improve: {improve} | Worsen: {worsen}",
                    "label": "Contradiction",
                })
        except (KeyError, TypeError):
            pass

        # 辅助3: ONTO关系
        try:
            onto_raw = meta["innovation_method"].get("onto", [])[:3]
            valid = [o for o in onto_raw if isinstance(o, list) and len(o) >= 3]
            if valid:
                text = " | ".join(f"{o[0]} -> {o[2]}" for o in valid)
                aux_onto.append({
                    "input_text": item["input_text"],
                    "target_text": text,
                    "label": "ONTO",
                })
        except (KeyError, TypeError):
            pass

    # --- 按比例采样辅助任务 ---
    all_samples = list(main_samples)
    ratio = config.AUX_SAMPLE_RATIO

    if ratio > 0 and (aux_fop or aux_contradiction or aux_onto):
        random.seed(42)

        def _sample(lst, r):
            n = max(1, int(len(lst) * r))
            return random.sample(lst, min(n, len(lst))) if lst else []

        s_fop = _sample(aux_fop, ratio)
        s_con = _sample(aux_contradiction, ratio)
        s_ont = _sample(aux_onto, ratio)

        all_samples.extend(s_fop)
        all_samples.extend(s_con)
        all_samples.extend(s_ont)

        print(f"\nDataset composition:")
        print(f"  Main (Solution):      {len(main_samples):>8,}")
        print(f"  Aux  (FOP):           {len(s_fop):>8,} / {len(aux_fop):,}")
        print(f"  Aux  (Contradiction): {len(s_con):>8,} / {len(aux_contradiction):,}")
        print(f"  Aux  (ONTO):          {len(s_ont):>8,} / {len(aux_onto):,}")

    print(f"  TOTAL:                {len(all_samples):>8,}")

    # --- 训练时间估算 ---
    micro_batches = len(all_samples) * config.EPOCHS // config.BATCH_SIZE
    est_low_h = micro_batches * 0.25 / 3600
    est_high_h = micro_batches * 0.5 / 3600
    opt_steps = micro_batches // config.GRAD_ACC

    print(f"\nTraining estimate (P100):")
    print(f"  Optimizer steps: ~{opt_steps:,}")
    print(f"  Time: {est_low_h:.1f} - {est_high_h:.1f} hours")
    if est_high_h > 10:
        print(f"  ** WARNING: may exceed Kaggle 12h limit! **")
        print(f"     Set AUX_SAMPLE_RATIO=0 or EPOCHS=1 to reduce")

    return Dataset.from_list(all_samples)


# ================= 3. Tokenization =================
def preprocess_fn(examples, tokenizer):
    inputs = tokenizer(
        examples["input_text"],
        max_length=config.MAX_SOURCE_LEN,
        truncation=True,
        padding=False,
    )
    labels = tokenizer(
        examples["target_text"],
        max_length=config.MAX_TARGET_LEN,
        truncation=True,
        padding=False,
    )["input_ids"]

    inputs["labels"] = [
        [(t if t != tokenizer.pad_token_id else -100) for t in lab]
        for lab in labels
    ]
    return inputs


# ================= 4. 语义空间提取 =================
def extract_and_save_semantic_space(model, tokenizer, device):
    """
    从原始训练数据(前N条)提取编码器语义向量.
    关键: 按行号顺序读取, 与metadata文件索引对齐.
    """
    print(f"\nExtracting semantic space (max {config.SEMANTIC_MAX_SAMPLES})...")
    model.eval()

    max_n = config.SEMANTIC_MAX_SAMPLES
    texts = []
    labels = []

    with open(config.TRAIN_FILE, "r", encoding="utf-8") as f:
        for line in f:
            if len(texts) >= max_n:
                break
            line = line.strip()
            if not line:
                break  # 遇到空行停止, 保持索引对齐
            try:
                item = json.loads(line)
                texts.append(item["input_text"])
                labels.append(item.get("label", ""))
            except json.JSONDecodeError:
                break  # 遇到解析错误停止, 保持索引对齐

    print(f"  Samples to encode: {len(texts)}")

    # 编码
    embeddings = []
    with torch.no_grad():
        for idx, text in enumerate(texts):
            inputs = tokenizer(
                text,
                max_length=config.MAX_SOURCE_LEN,
                truncation=True,
                return_tensors="pt",
            ).to(device)

            enc_out = model.get_encoder()(**inputs)
            emb = enc_out.last_hidden_state.mean(dim=1).cpu().numpy()[0]
            embeddings.append(emb)

            if (idx + 1) % 500 == 0:
                print(f"  Encoded: {idx + 1}/{len(texts)}")

    embeddings = np.array(embeddings)
    print(f"  Shape: {embeddings.shape}")

    # PCA降维
    n_comp = min(50, embeddings.shape[0] - 1, embeddings.shape[1])
    pca = PCA(n_components=n_comp, random_state=42)
    reduced = pca.fit_transform(embeddings)
    var_kept = float(np.sum(pca.explained_variance_ratio_))
    print(f"  PCA: {embeddings.shape[1]}d -> {n_comp}d, variance retained: {var_kept:.1%}")

    # 保存
    os.makedirs(config.SEMANTIC_DIR, exist_ok=True)

    with open(os.path.join(config.SEMANTIC_DIR, "pca_reducer.pkl"), "wb") as f:
        pickle.dump(pca, f)

    space_data = {
        "embeddings_full": embeddings.tolist(),
        "embeddings_reduced": reduced.tolist(),
        "labels": labels,
        "tasks": texts,
        "original_dim": int(embeddings.shape[1]),
        "reduced_dim": n_comp,
        "variance_retained": var_kept,
    }
    space_path = os.path.join(config.SEMANTIC_DIR, "semantic_space_pca.json")
    with open(space_path, "w", encoding="utf-8") as f:
        json.dump(space_data, f, ensure_ascii=False)

    print(f"  Saved: {space_path}")
    print(f"  Saved: pca_reducer.pkl")


# ================= 5. 主训练流程 =================
def train():
    # ------ GPU检查 ------
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Device: {device}")
    if device == "cuda":
        print(f"  GPU: {torch.cuda.get_device_name(0)}")
        mem_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"  VRAM: {mem_gb:.1f} GB")
    else:
        print("  WARNING: No GPU detected, training will be very slow")

    # ------ 加载模型 ------
    print(f"\nLoading model: {config.MODEL_NAME}")
    tokenizer = T5Tokenizer.from_pretrained(config.MODEL_NAME)
    model = T5ForConditionalGeneration.from_pretrained(config.MODEL_NAME)
    model = model.to(device)

    # ------ LoRA ------
    if config.USE_LORA:
        lora_cfg = LoraConfig(
            task_type=TaskType.SEQ_2_SEQ_LM,
            r=config.LORA_R,
            lora_alpha=config.LORA_ALPHA,
            lora_dropout=0.05,
            target_modules=["q", "v"],
        )
        model = get_peft_model(model, lora_cfg)
        model.print_trainable_parameters()

    # ------ 加载数据 ------
    raw_dataset = load_training_data()

    test_size = min(500, max(1, len(raw_dataset) // 100))
    dataset = raw_dataset.train_test_split(test_size=test_size, seed=42)

    print(f"\nTokenizing...")
    tokenized = dataset.map(
        lambda x: preprocess_fn(x, tokenizer),
        batched=True,
        batch_size=200,
        remove_columns=dataset["train"].column_names,
        desc="Tokenizing",
    )

    # ------ 训练参数 ------
    args = Seq2SeqTrainingArguments(
        output_dir=config.OUTPUT_DIR,
        num_train_epochs=config.EPOCHS,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRAD_ACC,
        learning_rate=config.LR,
        warmup_steps=config.WARMUP_STEPS,
        weight_decay=0.01,
        fp16=(device == "cuda"),
        eval_strategy="steps",
        eval_steps=2000,
        save_strategy="steps",
        save_steps=2000,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        logging_steps=50,
        report_to="none",
        dataloader_num_workers=2,
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"],
        tokenizer=tokenizer,
        data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    )

    # ------ 训练 ------
    print("\n" + "=" * 60)
    print("Training started")
    print("=" * 60)
    trainer.train()

    # ------ 合并LoRA并保存完整模型 ------
    print("\n" + "=" * 60)
    print("Saving model")
    print("=" * 60)

    if config.USE_LORA:
        print("  Merging LoRA weights into base model...")
        model = model.merge_and_unload()
        # model 现在是普通 T5ForConditionalGeneration

    os.makedirs(config.FINAL_MODEL_DIR, exist_ok=True)
    model.save_pretrained(config.FINAL_MODEL_DIR)
    tokenizer.save_pretrained(config.FINAL_MODEL_DIR)
    print(f"  Model saved: {config.FINAL_MODEL_DIR}")

    # 验证模型可加载
    print("  Verifying model is loadable...")
    _test = T5ForConditionalGeneration.from_pretrained(config.FINAL_MODEL_DIR)
    del _test
    print("  OK: model loads as T5ForConditionalGeneration")

    # ------ 快速生成测试 ------
    print("\nQuick generation test:")
    model.eval()
    test_input = tokenizer(
        "[Domain: B25J] Task: reduce vibration | Direction: Decrease",
        return_tensors="pt",
        max_length=config.MAX_SOURCE_LEN,
        truncation=True,
    ).to(device)
    with torch.no_grad():
        out = model.generate(**test_input, max_length=64, num_beams=3)
    print(f"  Output: {tokenizer.decode(out[0], skip_special_tokens=True)[:200]}")

    # ------ 语义空间提取 (训练后) ------
    print("\n" + "=" * 60)
    print("Semantic space extraction (post-training)")
    print("=" * 60)

    if device == "cuda":
        torch.cuda.empty_cache()

    extract_and_save_semantic_space(model, tokenizer, device)

    # ------ 完成 ------
    print("\n" + "=" * 60)
    print("ALL DONE")
    print("=" * 60)
    print(f"  Model:          {config.FINAL_MODEL_DIR}")
    print(f"  Semantic space: {config.SEMANTIC_DIR}")
    print(f"\nNext steps:")
    print(f"  1. Download final_model/ and semantic_analysis/ from Kaggle output")
    print(f"  2. Run manifold_analysis.py for mathematical proofs")
    print(f"  3. Deploy inference_service.py")


if __name__ == "__main__":
    train()


2026-02-11 08:11:34.427030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770797494.818891      23 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770797494.963373      23 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770797495.793119      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770797495.793160      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770797495.793163      23 computation_placer.cc:177] computation placer alr

Device: cuda
  GPU: Tesla P100-PCIE-16GB
  VRAM: 15.9 GB

Loading model: Langboat/mengzi-t5-base


spiece.model:   0%|          | 0.00/725k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

Tokenizing:   0%|          | 0/500 [00:00<?, ? examples/s]

/tmp/ipykernel_23/2471617983.py:428: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(



Training started


Step,Training Loss,Validation Loss
2000,3.147300,2.789413
4000,2.882600,2.619256
6000,2.807100,2.552227
8000,2.758700,2.512638
10000,2.774700,2.487984
12000,2.692700,2.473910



Saving model
  Merging LoRA weights into base model...
  Model saved: /kaggle/working/final_model
  Verifying model is loadable...
  OK: model loads as T5ForConditionalGeneration

Quick generation test:
  Output: Improve: 操作性能 | Worsen: 结构复杂度

Semantic space extraction (post-training)

Extracting semantic space (max 5000)...
  Samples to encode: 5000
  Encoded: 500/5000
  Encoded: 1000/5000
  Encoded: 1500/5000
  Encoded: 2000/5000
  Encoded: 2500/5000
  Encoded: 3000/5000
  Encoded: 3500/5000
  Encoded: 4000/5000
  Encoded: 4500/5000
  Encoded: 5000/5000
  Shape: (5000, 768)
  PCA: 768d -> 50d, variance retained: 83.5%
  Saved: /kaggle/working/semantic_analysis/semantic_space_pca.json
  Saved: pca_reducer.pkl

ALL DONE
  Model:          /kaggle/working/final_model
  Semantic space: /kaggle/working/semantic_analysis

Next steps:
  1. Download final_model/ and semantic_analysis/ from Kaggle output
  2. Run manifold_analysis.py for mathematical proofs
  3. Deploy inference_service.py


In [6]:
# from transformers import T5Tokenizer
# import json
# import matplotlib.pyplot as plt

# tokenizer = T5Tokenizer.from_pretrained("Langboat/mengzi-t5-base")

# input_lengths = []
# target_lengths = []

# with open("/kaggle/input/datasets/yyk22004396/universal-innovation-training-set/kaggle_upload/patent_expert_final.jsonl", "r") as f:
#     for i, line in enumerate(f):
#         if i >= 10000:  # 抽样1万条足够
#             break
#         data = json.loads(line)
#         input_lengths.append(len(tokenizer(data["input_text"])["input_ids"]))
#         target_lengths.append(len(tokenizer(data["target_text"])["input_ids"]))

# # 打印关键分位数
# print("Input lengths (tokens):")
# print(f"  90% ≤ {sorted(input_lengths)[int(0.9*len(input_lengths))]}")
# print(f"  95% ≤ {sorted(input_lengths)[int(0.95*len(input_lengths))]}")
# print(f"  max = {max(input_lengths)}")

# print("\nTarget lengths (tokens):")
# print(f"  90% ≤ {sorted(target_lengths)[int(0.9*len(target_lengths))]}")
# print(f"  95% ≤ {sorted(target_lengths)[int(0.95*len(target_lengths))]}")
# print(f"  max = {max(target_lengths)}")